In [1]:
import pandas as pd
import plotly.express as px
import tqdm
import re
import plotly.graph_objects as go
import glob

# Ergebnisse als Dataframe einlesen

In [2]:
df = pd.read_csv("data/roman_corpus_word_frequencies.csv", sep=";")

In [3]:
df_year = df.groupby(["Jahr"]).mean().reset_index()
df_year_median = df.groupby(["Jahr"]).median().reset_index()

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_1237/3862513688.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_year = df.groupby(["Jahr"]).mean().reset_index()
/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_1237/3862513688.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_year_median = df.groupby(["Jahr"]).median().reset_index()


# Allgemein: Plots zum Anteil konkreter und abstrakter Wörter

In [8]:
# Line-Plot für anteil konkreter Wörter

df_year["MA_konkret"] = df_year["anteil_konkret"].rolling(window=3).mean()

fig = px.line(df_year, x="Jahr", y=["anteil_konkret"], color_discrete_sequence=["lightgrey"])
fig.add_trace(go.Scatter(x=df_year.Jahr, 
                         y=df_year["MA_konkret"], 
                         opacity=0.7, 
                         line=dict(color='darkgreen', width=2), 
                         name='geglättet'))
fig.write_image("figures/lineplot_anteil_konkret.png")

In [9]:
# Line-Plot für anteil abstrakter Wörter

df_year["MA_abstrakt"] = df_year["anteil_abstrakt"].rolling(window=3).mean()

fig = px.line(df_year, x="Jahr", y=["anteil_abstrakt"], color_discrete_sequence=["lightgrey"])
fig.add_trace(go.Scatter(x=df_year.Jahr, 
                         y=df_year["MA_abstrakt"], 
                         opacity=0.7, 
                         line=dict(color='darkblue', width=2), 
                         name='geglättet'))

fig.write_image("figures/lineplot_anteil_abstrakt.png")

In [59]:
# Sactterplot mit Anteil konkreter Wörter pro Jahr
fig = px.scatter(df, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                 trendline="ols", trendline_options=dict(log_x=True), trendline_color_override="black",\
                 color_discrete_sequence=["darkred"],\
                )
fig.write_image("figures/scatterplot_anteil_konkret.png")

In [64]:
# Sactterplot mit Anteil konkreter Wörter pro Jahr, colored nach authorgender
fig = px.scatter(df, x="Jahr", y="anteil_konkret", color="Geschlecht",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                 trendline="ols",\
                 color_discrete_sequence=["orange", "indianred"],\
                )
fig.write_image("figures/scatterplot_anteil_konkret_col_gender.png")

In [61]:
# Sactterplot mit Anteil abstrakter Wörter pro Jahr
fig = px.scatter(df, x="Jahr", y="anteil_abstrakt",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                 trendline="ols", trendline_options=dict(log_x=True), trendline_color_override="black",\
                )
fig.write_image("figures/scatterplot_anteil_abstrakt.png")

In [63]:
# Sactterplot mit Anteil abstrakter Wörter pro Jahr, colored nach authorgender
fig = px.scatter(df, x="Jahr", y="anteil_abstrakt", color="Geschlecht",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                 trendline="ols",\
                 color_discrete_sequence=["orange", "indianred"],\
                )
fig.write_image("figures/scatterplot_anteil_abstrakt_col_gender.png")

In [4]:
# Sactterplot, in dem Anteil konkreter Wörter gegen Anteil abstrakter Wörter geplottet wird + coloriert nach Jahr
fig = px.scatter(df, x="anteil_konkret", y="anteil_abstrakt", color="Jahr",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                color_continuous_scale=px.colors.sequential.Plotly3,\
                )
fig.write_image("figures/scatterplot_anteil_abstrakt_vs_konkret_col_year.png")

In [6]:
# Sactterplot, in dem Anteil konkreter Wörter gegen Anteil abstrakter Wörter geplottet wird + coloriert nach authorgender
fig = px.scatter(df, x="anteil_konkret", y="anteil_abstrakt", color="Geschlecht",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                color_continuous_scale=px.colors.sequential.Plotly3,\
                 color_discrete_sequence=["orange", "indianred"],\
                 trendline="ols",\
                )
fig.write_image("figures/scatterplot_anteil_abstrakt_vs_konkret_col_gender.png")

# Lineplots für einzelne Wortfelder

In [5]:
wortfelder_df = pd.read_csv("data/wortfelder.csv", sep=";", encoding="utf-8")
wortfelder = wortfelder_df.Wortfeld.to_list()

In [22]:
for wortfeld in wortfelder:
    MA = "MA" + wortfeld
    anteil = "anteil_" + wortfeld
    df_year[MA] = df_year[anteil].rolling(window=3).mean()
    #filename = "line_plots/line_plot_" + wortfeld + ".png"
    fig = px.line(df_year, x="Jahr", y=anteil, color_discrete_sequence=["lightgrey"])
    fig.add_trace(go.Scatter(x=df_year.Jahr, 
                         y=df_year[MA], 
                         opacity=0.7, 
                         line=dict(color='darkblue', width=2), 
                         name='geglättet'))
    fig.write_image("figures/lineplots_einzelne_wortfelder/" + str(wortfeld) + ".png")

# Lineplot, in dem Kurven verschiedener Wortfelder übereinander liegen.

In [24]:
# konkrete Wortfelder
fig = px.line(df_year, x="Jahr", y=["anteil_Koerperteile", "anteil_Gebaeudeteile", "anteil_Bauwerke",\
                                    "anteil_Farben", "anteil_Kleidung", "anteil_helligkeitsspezifisch",\
                                    "anteil_Pflanzen", "anteil_Moebel", "anteil_Zimmer", "anteil_aussehensspezifisch",\
                                    "anteil_Kleidungsteile", "anteil_Heimtextilien", "anteil_Haushaltsgegenstaende",\
                                    "anteil_Dekorationsgegenstaende", "anteil_Aufmachung", "anteil_Taschen", "anteil_Stoffe",\
                                    "anteil_formspezifisch", "anteil_oberflaechenspezifisch",\
                                   "anteil_Holz", "anteil_Waelder", "anteil_Gartenanlagen", "anteil_Gruenflaechen", "anteil_Kunstobjekte",\
                                    "anteil_Bilder", "anteil_Ornamente"])

fig.update_layout(
    
    legend=dict(x=-0.01, y=-0.2),
    legend_orientation="h")
fig.update_layout(title_font_family="Times New Roman", legend_title_text='Wortfelder')

fig.write_image("figures/lineplot_anteil_konkret_mehrere_wortfelder.png")

In [25]:
# konkrete und abstrakte Wortfelder
fig = px.line(df_year, x="Jahr", y=["anteil_Emotionen", "anteil_Charaktereigenschaften", "anteil_Koerperteile", "anteil_Gebaeudeteile", "anteil_Bauwerke",\
                                    "anteil_Farben", "anteil_Kleidung", "anteil_helligkeitsspezifisch",\
                                    "anteil_Pflanzen", "anteil_Moebel", "anteil_Zimmer", "anteil_aussehensspezifisch",\
                                    "anteil_Kleidungsteile", "anteil_Heimtextilien", "anteil_Haushaltsgegenstaende",\
                                    "anteil_Dekorationsgegenstaende", "anteil_Aufmachung", "anteil_Taschen", "anteil_Stoffe",\
                                    "anteil_formspezifisch", "anteil_oberflaechenspezifisch",\
                                   "anteil_Holz", "anteil_Waelder", "anteil_Gartenanlagen", "anteil_Gruenflaechen", "anteil_Kunstobjekte",\
                                    "anteil_Bilder", "anteil_Ornamente"])

fig.update_layout(
    
    legend=dict(x=-0.01, y=-0.2),
    legend_orientation="h")
fig.update_layout(title_font_family="Times New Roman", legend_title_text='Wortfelder')

fig.write_image("figures/lineplot_anteil_konkret_u_abstrakt_mehrere_wortfelder.png")

# Plots für Wörter aus Bereich "Figuren"

In [30]:
df["Anteil Wörter Figurenaussehen"] = df["anteil_Koerperteile"] + df["anteil_Kleidung"] + df["anteil_Kleidungsteile"] + df["anteil_Schmuck"] + df["anteil_Taschen"] + df["anteil_Aufmachung"] + df["anteil_aussehensspezifisch"]

In [32]:
fig = px.scatter(df, y="Anteil Wörter Figurenaussehen", x="Jahr",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                
                 trendline="ols",\
                 color_discrete_sequence=["purple"]
                
                )
fig.data[1].update(line_color='black') 

fig.write_image("figures/scatterplot_anteil_figurenaussehen.png")

In [33]:
# Figurenaussehen und Autor
df_special = df.copy()
hervorheben = ["Dauthendey, Max", "Mereau, Sophie"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3600/3063989296.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
fig = px.scatter(df_special, y="Anteil Wörter Figurenaussehen", x="Jahr",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"],\
                
                 trendline="ols",
                color="Autor_normalisiert"
                )
fig.data[1].update(line_color='black') 
#fig.update_layout({"plot_bgcolor": "rgba(0, 0, 0, 0)", "paper_bgcolor": "rgba(0, 0, 0, 0)",})

fig.write_image("figures/scatterplot_anteil_figurenaussehen_dauthendey_mereau.png")

# Plots für Häufigkeiten unter Berücksichtigung der Redewiedergabe-Form


In [3]:
df = pd.read_csv("data/roman_korpus_word_frequencies_redewiedergabe.csv", sep=";")

In [38]:
df_year = df.groupby(["Jahr"]).mean().reset_index()
df_year_median = df.groupby(["Jahr"]).median().reset_index()

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3600/3862513688.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3600/3862513688.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [39]:
df_year["MA_direct"] = df_year["ant_dir_rede"].rolling(window=3).mean()
df_year["MA_indirect"] = df_year["ant_ind_rede"].rolling(window=3).mean()
df_year["MA_reported"] = df_year["ant_rep"].rolling(window=3).mean()

## Häufigkeiten der Redetypen

In [49]:
# Anteile Redewiedergabe Typen
fig = px.line(df_year, x="Jahr", y=["ant_dir_rede", "ant_ind_rede", "ant_rep"])

fig.update_layout(
    
    legend=dict(x=-0.01, y=-0.2),
    legend_orientation="h")
fig.update_layout(title_font_family="Times New Roman", legend_title_text='Wortfelder')

fig.write_image("figures/redewiedergabe/lineplot_anteil_redewiedergabe.png")

## Verteilung konkret/abstrakt in Bezug auf Redewiedergabe

In [50]:
# Anteil konkreter Wörter an Redewiedergabe-Typ: Erzählerrede
fig = px.scatter(df, x="Jahr", y="ant_narr_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename", "ant_narr_konkret"],\
                trendline="ols", trendline_options=dict(log_x=True), trendline_color_override="black",\
                color_discrete_sequence=["darkred"])
                 #color="Titel",\
                 #color_discrete_sequence=["lightblue", "orange", "green", "red", \
                                          #"darkgreen","midnightblue", "darkmagenta", "pink", "magenta"])
                
fig.write_image("figures/redewiedergabe/scatterplot_anteil_konkret_erzählerrede.png")

In [54]:
# Anteil abstrakter Wörter an Redewiedergabe-Typ: Erzählerrede

fig = px.scatter(df, y="ant_narr_abstrakt", x="Jahr",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 trendline="ols", trendline_options=dict(log_x=True), trendline_color_override="black",\
                 #color_discrete_sequence=["lightblue", "orange", "green", "red", "darkgreen","midnightblue", "darkmagenta", "pink", "magenta"],\
                
                )
fig.write_image("figures/redewiedergabe/scatterplot_anteil_abstrakt_erzählerrede.png")

# Plots einfärben nach Autoren

In [19]:
# Theodor Fontane
df_special = df.copy()
hervorheben = ["Fontane, Theodor"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "orange"],\
                
                )


fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_fontane.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/4028528610.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
# Friedrich Spielhagen
df_special = df.copy()
hervorheben = ["Spielhagen, Friedrich"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "green"],\
                
                )


fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_spielhagen.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/3264129736.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [21]:
# Eugenie Marlitt
df_special = df.copy()
hervorheben = ["Marlitt, Eugenie"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "red"],\
                
                )

fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_marlitt.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/8852746.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
# Wieland, Christoph Martin
df_special = df.copy()
hervorheben = ["Wieland, Christoph Martin"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "midnightblue"],\
                
                )

fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_wieland.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/3082970846.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [24]:
# Dauthendey, Max
df_special = df.copy()
hervorheben = ["Dauthendey, Max"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "darkmagenta"],\
                
                )

fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_dauthendey.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/1969162249.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [27]:
# Stifter, Adalbert
df_special = df.copy()
hervorheben = ["Stifter, Adalbert"]
for i in range(len(df_special)):
    if df_special["Autor_normalisiert"].loc[i] not in hervorheben:
        df_special["Autor_normalisiert"].loc[i] = "other"
        #df_special["Titel"].loc[i] = "other"

    
fig = px.scatter(df_special, x="Jahr", y="anteil_konkret",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], \
                 color="Autor_normalisiert",\
                 trendline="ols",\
                 color_discrete_sequence=["lightblue", "magenta"],\
                
                )

fig.write_image("figures/colored_autoren/scatterplot_anteil_konkret_col_stifter.png")

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_4676/538197966.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Konkretheitswerte insgesamt (Köper/Schulte im Walde)

In [69]:
dfac = pd.read_csv("roman_corpus_AbstConc.csv", sep=";")
dfac = dfac.rename(columns={"anteil_AbstConc": "Konkretheit"})

In [70]:
dfac_year = dfac.groupby(["Jahr"]).mean().reset_index()
dfac_year_median = dfac.groupby(["Jahr"]).median().reset_index()
#dfac_year_median.tail()

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3600/3334473707.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/3c/d5qkn8zs6f9bq34fzlrqkm780000gn/T/ipykernel_3600/3334473707.py:2: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [72]:
dfac_year["MA_anteil_AbstConc"] = dfac_year["Konkretheit"].rolling(window=3).mean()

In [76]:
# anteil_konkret vs anteil_abstrakt
fig = px.scatter(dfac, y="Konkretheit", x="Jahr",\
                 hover_data=["Titel", "Autor_normalisiert", "filename"], trendline="ols",\
                trendline_options=dict(log_x=True), trendline_color_override="black", \
                 color_discrete_sequence=["black"]
                )
fig.write_image("figures/scatterplot_konkretheitswerte.png")